# Seamless 2

https://github.com/facebookresearch/seamless_communication

https://huggingface.co/facebook/seamless-m4t-v2-large

https://huggingface.co/facebook/seamless-m4t-v2-large#transformers-usage

https://huggingface.co/docs/transformers/main/en/model_doc/seamless_m4t_v2

In [1]:
pip install git+https://github.com/huggingface/transformers.git sentencepiece

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-axr2uh_9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-axr2uh_9
  Resolved https://github.com/huggingface/transformers.git to commit a502b0d427e6ea217bb4d28b352297823385860a
  Installing build dependencies ... - \ | / - done
  Getting requirements to build wheel ... - \ | done
  Preparing metadata (pyproject.toml) ... - \ | done
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/05/09/1945ca6ba3ad8ad6e2872ba682ce8d68c5e63c8e55458ed8ab4885709f1d/huggingface_hub-0.19.4-py3-none-any.whl.metadata
INFO: pip is looking at multiple versions of tokenizers to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for tokenizers<0.19,>=0.14 from https://files.pythonhosted.org/packages/eb/3d/ee

In [2]:
from transformers import AutoProcessor, SeamlessM4Tv2Model
import torchaudio
from IPython.display import Audio

processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large")
model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large")

/opt/conda/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/19.7k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.17M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/211k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.24G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/9.91M [00:00<?, ?B/s]

## TEXT -> TRANSLATE -> SPEECH

In [3]:
text_inputs = processor(text = "Hello, my dog is cute", src_lang="eng", return_tensors="pt")
audio_array_from_text = model.generate(**text_inputs, tgt_lang="arb")[0].cpu().numpy().squeeze()
print( audio_array_from_text )


sample_rate = model.config.sampling_rate
Audio(audio_array_from_text, rate=sample_rate)

[-3.2775220e-04 -2.7725974e-04 -2.5808116e-04 ... -8.3542109e-05
 -1.3815724e-04 -7.8639168e-05]


## SPEECH -> TRANSLATE -> SPEECH

In [4]:
# from audio
audio, orig_freq =  torchaudio.load("https://www2.cs.uic.edu/~i101/SoundFiles/preamble10.wav")
audio =  torchaudio.functional.resample(audio, orig_freq=orig_freq, new_freq=16_000) # must be a 16 kHz waveform array
audio_inputs = processor(audios=audio, return_tensors="pt")
audio_array_from_audio = model.generate(**audio_inputs, tgt_lang="rus")[0].cpu().numpy().squeeze()

print(audio_array_from_audio)
Audio(audio_array_from_audio, rate=sample_rate)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[-1.3429482e-04 -5.3192147e-05 -6.2640094e-05 ...  1.3625615e-04
  9.6801516e-05  1.9601602e-04]


## SPEECH -> TRANSLATE -> TEXT

In [5]:
audio, orig_freq =  torchaudio.load("https://www2.cs.uic.edu/~i101/SoundFiles/preamble10.wav")
audio_inputs = processor(audios=audio, return_tensors="pt")
output_tokens = model.generate(**audio_inputs, tgt_lang="ces", generate_speech=False)
translated_text_from_audio = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
print(translated_text_from_audio)

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


my, lidé Spojených států, abychom vytvořili dokonalější unii, vytvořili spravedlnost, zajistili domácí klid, zajišťovali společnou obranu,


## TEXT -> TRANSLATE -> TEXT

In [6]:
text_inputs = processor(text = "Hello, my dog is cute", src_lang="eng", return_tensors="pt")
output_tokens = model.generate(**text_inputs, tgt_lang="ces", generate_speech=False)
translated_text_from_text = processor.decode(output_tokens[0].tolist()[0], skip_special_tokens=True)
print(translated_text_from_text)

Ahoj, můj pes je roztomilý.
